## Spyglass tutorial 0

**Note: make a copy of this notebook and run the copy to avoid git conflicts in the future**

This is the zeroth in a multi-part tutorial on the Spyglass pipeline used in Loren Frank's lab, UCSF. It gives a general introduction to Datajoint and shows how to insert an NWB file into the Datajoint database, inspect and query the data, and delete it.

Let's start by importing the `spyglass` package, along with a few others. 

In [ ]:
import os
import numpy as np

import spyglass as sg
import datajoint as dj

# ignore datajoint+jupyter async warnings
import warnings

warnings.simplefilter("ignore", category=DeprecationWarning)
warnings.simplefilter("ignore", category=ResourceWarning)
warnings.simplefilter("ignore", category=UserWarning)


In [ ]:
# We also import spyglass.common so we can call tables more easily
import spyglass.common as sgc

# We import spyglass.data_import to allow for inserting an NWB file into the database
import spyglass.data_import as sdi


### Visualizing the database

Datajoint enables users to use Python to build and interact with a *Relational Database*. A *Relational Database* stores and provides access to data stored in row-features and column-features. Each row provide information about different features of an individivial object while each column provide information about a single feature for a collection of objects. It supports the ability to create a new object, read an existing object, update an existing object and delete an object.

The following diagram (called *entity relationship diagram*) shows all the tables in our database and their relationships.

Polygons are tables:
* Blue oval: tables whose entries are imported from external files (e.g. NWB file).
* Green rectangle: tables whose entries are entered manually.
* Red circle: tables whose entries are computed from entries of other tables.
* No shape (only text): tables whose entries are part of the table upstream

Lines are *dependencies* between tables:
* Bold lines: downstream table inherits the *primary key* ^ of upstream table as its sole primary key
* Solid lines: downstream table inherits the primary key of upstream table as part of its primary key
* Dashed lines: downstream table inherits the primary key of upstream table as non-primary key

^ Primary key: a set of attributes (i.e. column names) used to uniquely define an entry (i.e. a row)

In [ ]:
# Draw tables that are three levels below and one level above Session
dj.ERD(sgc.Session) - 1 + 3


### Example data

Now that we have a general idea of how our database is organized, we will try inserting new data to it. We assume that the data is a neural recording (along with other auxiliary data) that has already been converted to the NWB format. For the purposes of this tutorial, we will use `montague20200802.nwb`. If you're accessing the Frank lab database on the UCSF network, this file can be found in `/stelmo/nwb/raw` directory (assuming you have mounted `stelmo` at `/`). If you do not have access to Frank lab database, then download `montague20200802.nwb` from [here](https://www.dropbox.com/s/srra33e0igaj5rm/montague20200802.nwb?dl=0) - it is rather big (~ 70 GB) so be aware of that. 

Once you have the NWB file, you should copy it and rename the copy to something unique (e.g. `montague20200802_yourname.nwb`). This is because many people will be using this example file to practice inserting data, and a file can only be inserted once (the file name acts as a *primary key* in the `Session` table). 

In [ ]:
# Define the name of the file that you copied and renamed; make sure it's something unique.
nwb_file_name = "montague20200802_tutorial.nwb"
filename, file_extension = os.path.splitext(nwb_file_name)
# This is a copy of the original nwb file, except it doesn't contain the raw data (for storage reasons)
nwb_copy_file_name = filename + "_" + file_extension


In [ ]:
nwb_file_name


### Inserting data

As you may know, the NWB file contains a lot of information, such as information about the experimenter (e.g. who did the experiment, where was it done, etc); the animal's behavior (e.g. video recording of the animal's position in an environment during the experiment); the neural activity (extracellular recording of multiple brain areas) etc. We wish to enter this information into the tables of our Datajoint database so that we can easily access them later. If we have an NWB file that has been properly generated, this is straightforward: just run the `sg.insert_session` function, which will populate many of the tables automatically. Let's do this for our example NWB file.

Note: this may take a while because it makes a copy of the NWB file.

In [ ]:
sdi.insert_sessions(nwb_file_name)


### Inspecting and querying data

To look at the tables, just call it (don't forget the `()` at the end, as tables are like Python classes). Let's try calling the `Lab` table.

In [ ]:
sgc.Lab()


There is only one attribute (`lab_name`) and two entries (`Loren Frank`, `Giocomo`) in this table. Note that even though `Lab` is one of the tables that has to be manually entered (i.e. the green rectangles in our ERD), `sg.insert_sessions` populates it from the NWB file.

Let's look at a more interesting table: `Session`.

In [ ]:
sgc.Session()


This session has multiple attributes. The attributes that make up the primary key are shown in bold. In this case, there is only one attribute in the primary key: `nwb_file_name`. Given that many downstream tables are connected to `Session` via bold lines, you can use this attribute to uniquely define data entries in many tables in our database.

In [ ]:
# can also look at the docstring
sgc.Session.describe()


To look at specific entries from a table, include the appropriate condition with the `&` operator. The condition must be in the form of a key-value pair, or a dictionary. As an example, let's view only the entry whose `nwb_file_name` is our example NWB file in `Session`:

In [ ]:
sgc.Session & {"nwb_file_name": nwb_copy_file_name}


Now let's go to a downstream table: `Raw`. This table is connected to `Session` table with a bold line, so it has the same primary key.

In [ ]:
sgc.Raw & {"nwb_file_name": nwb_copy_file_name}


`IntervalList` table is connected to `Session` table with a solid line, as it inherits `nwb_file_name` as one of the two attributes that make up the primary key. This means that you need to know both `nwb_file_name` and `interval_list_name` to uniquely identify an entry.

In [ ]:
sgc.IntervalList & {"nwb_file_name": nwb_copy_file_name}


As you can see, some of the data is shown in the table (e.g. `interval_list_name`) while others are obscured (e.g. `valid_times` are shown as `=BLOB=`). To inspect the data, use the `fetch` (for getting multiple entries) or `fetch1` (for getting just one entry) methods. The following query returns `valid_times` of an interval list called `04_r2`.

In [ ]:
(
    sgc.IntervalList
    & {"nwb_file_name": nwb_copy_file_name, "interval_list_name": "04_r2"}
).fetch1("valid_times")


Query supports many operations and conditions - just make sure the conditions are in the form of a dictionary. We have already seen that `&` represents the set-theoretic intersection. `-`, on the other hand, is like the set-theoretic complement. The following query returns all `interval_list_name` that is not `01_s1` or `04_r2`.

In [ ]:
(
    (
        (sgc.IntervalList & {"nwb_file_name": nwb_copy_file_name})
        - {"interval_list_name": "01_s1"}
    )
    - {"interval_list_name": "04_r2"}
).fetch("interval_list_name")


Other available operations are described [here](https://docs.datajoint.org/python/queries/05-Operators.html). Also feel free to inspect other tables and learn what kind of data they contain.

### Deleting data 

Another neat feature of Datajoint is that it automatically maintains the integrity of your data. For example, if we were to delete our entry in the `Session` table, the associated entries in all of the downstream tables (e.g. `Raw`, `IntervalList`, `ElectrodeGroup` to name a few) will also be deleted. That way, there is no 'orphan' data whose origin cannot be tracked. We will do this now using the `delete` method.

In [ ]:
# our data is currently in Session table
sgc.Session & {"nwb_file_name": nwb_copy_file_name}


In [ ]:
# Type `yes` when prompted to delete
(sgc.Session & {"nwb_file_name": nwb_copy_file_name}).delete()


In [ ]:
# Check that delete worked
sgc.Session & {"nwb_file_name": nwb_copy_file_name}


`delete` method is useful when you need to re-do something. We discourage editing an entry because this will affect the associated entries in downstream tables and violate data integrity. Instead it is recommended that you just delete and re-enter it. This is easy to do, as most data entry is automated in our pipeline.

In [ ]:
# Entries are also gone from downstream tables, e.g. IntervalList
sgc.IntervalList & {"nwb_file_name": nwb_copy_file_name}


We're not quite done yet. Not only should we delete our entry from `Session`, we should also delete the associated entry in `Nwbfile`. Recall from the entity relationship diagram that `Nwbfile` is upstream of `Session`. That means getting rid of our entry from `Session` doesn't affect the corresponding entry in `Nwbfile` - this has to be manually removed. To do so, we again use the `delete` method, but in the case of `Nwbfile` there is an extra step: we need to remove the NWB file itself as well as the entry from `Nwbfile` table. To remove the files, we run the `cleanup` method with the `delete_files` argument as `True`. 

Note that the same idea applies to deleting files from `AnalysisNwbfile` table. 

In [ ]:
# Check out the Nwb file
sgc.Nwbfile & {"nwb_file_name": nwb_copy_file_name}


In [ ]:
# Let's delete the entry
(sgc.Nwbfile & {"nwb_file_name": nwb_copy_file_name}).delete()


In [ ]:
# Note that the file (ends with _.nwb) has not been deleted, even though the entry is
!ls $SPYGLASS_BASE_DIR/raw

In [ ]:
# We clean it up
sgc.Nwbfile().cleanup(delete_files=True)


In [ ]:
# Now the file is gone as well
!ls $SPYGLASS_BASE_DIR/raw

### Creating a Lab Team

Before we end this tutorial, there is one last thing we must do: create a lab team. A lab team is a set of lab members who own a set of NWB files and the associated information in our Datajoint database. Think of this as a "subgroup" within the lab that collaborates on the same projects. Only the members of a lab team will be able to delete table entries they made (this permission system is not yet implemented; we're working on it). Right now we need to set a lab team so that the permission for manual curation of spike sorted data can be set (this will be covered in [tutorial 2](2_curation.ipynb)).

The `LabMember` table contains the list of lab members. It has a parts table called `LabMemberInfo`, where the Google account of each member can be found (for authentication purposes). Similarly, `LabTeam` table contains the list of lab teams and has a parts table called `LabTeamMember` which specifies which of the lab members (as entered in `LabMember`) belongs to each lab team. Both `LabMember` and `LabTeam` are `dj.Manual` tables, which means the data can be entered manually (although when we add our NWB file with `nd.insert_sessions(nwb_file_name)`, the owner of the NWB file (as specified in the `experimenter` field) is automatically entered into `LabMember` table). 

To proceed, we will create a new team using the `create_new_team` method of `LabTeam`. You will give this team a name, and add yourself (and your Google account) to this team. Later, when we do spike sorting, we will specify the team the sorting belongs to. This will give everyone in the team (in this case, just you) the permission to add curation labels.

In [ ]:
# take a look at the lab members
sgc.LabMember()


In [ ]:
# LabMember also has a Parts table called LabMemberInfo
sgc.LabMember.LabMemberInfo()


In [ ]:
# these are the existing lab teams
sgc.LabTeam()


In [ ]:
# create a new team
# change team_name to something unique
# change team_members
# team_description is optional
sgc.LabTeam().create_new_team(
    team_name="Beans", team_members=["Alison Comrie"], team_description="test"
)


In [ ]:
# add info about the team members
# add your name and your google account
sgc.LabMember.LabMemberInfo.insert(
    [["Alison Comrie", "comrie.alison@gmail.com", "alison"]], skip_duplicates=True
)


In [ ]:
sgc.LabMember.LabMemberInfo()
